# Pytorch 1.11버전 이상, 파이썬 3.9버전 이상 사용해주세요!

In [1]:
!python --version


Python 3.10.12


In [2]:
!pip install sentence_transformers

In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
    'The cat jumps over the small wooden fence',
    'She enjoys reading books under the tall tree',
    'A plane flies high across the clear blue sky',
    'They play soccer together every weekend after school',
    'The sun rises early in the quiet mountain village',
    'He bought fresh bread from the bakery this morning',
    'A dog barked loudly at the stranger passing by',
    'We visited a museum filled with ancient historical artifacts',
    'She painted a colorful sunset on the wide canvas',
    'The kids built a sandcastle near the ocean',
]

In [17]:
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)

(12, 384)


In [18]:
similarities = model.similarity(embeddings, embeddings)
print(similarities)

tensor([[ 1.0000,  0.6660,  0.0150,  0.1456,  0.2039, -0.0053,  0.3939,  0.0994,
          0.0866,  0.0231,  0.3335,  0.0012],
        [ 0.6660,  1.0000,  0.0957,  0.1677,  0.2237,  0.0476,  0.4303,  0.0563,
          0.1325, -0.0300,  0.3868,  0.0564],
        [ 0.0150,  0.0957,  1.0000,  0.1147,  0.1214,  0.1179,  0.0189,  0.1890,
          0.2259,  0.0593,  0.0812, -0.0151],
        [ 0.1456,  0.1677,  0.1147,  1.0000,  0.0637,  0.0402,  0.2205,  0.0449,
          0.0891,  0.1233,  0.1408,  0.0893],
        [ 0.2039,  0.2237,  0.1214,  0.0637,  1.0000,  0.0217,  0.1154,  0.0829,
          0.0689, -0.0202,  0.2278,  0.0585],
        [-0.0053,  0.0476,  0.1179,  0.0402,  0.0217,  1.0000,  0.0058,  0.1295,
         -0.0146, -0.0268,  0.0047,  0.1559],
        [ 0.3939,  0.4303,  0.0189,  0.2205,  0.1154,  0.0058,  1.0000,  0.0719,
          0.1048,  0.0625,  0.2684,  0.1087],
        [ 0.0994,  0.0563,  0.1890,  0.0449,  0.0829,  0.1295,  0.0719,  1.0000,
          0.0688,  0.0117, -0.

##FAISS

In [8]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00


In [14]:
import faiss
import numpy as np

In [19]:
dimension = embeddings.shape[1]
print(f"Dimension of the embeddings: {dimension}")
index = faiss.IndexFlatL2(dimension)  # Using L2 distance metric
index.add(embeddings)  # Add embeddings to the index

Dimension of the embeddings: 384


In [20]:
print("Total vectors in FAISS index:", index.ntotal)


Total vectors in FAISS index: 12


In [21]:
# 검색할 쿼리 문장을 임베딩으로 변환
query = model.encode(["lovely day"])  # 쿼리 문장
query_np = np.array(query).astype('float32')

# 상위 2개의 유사 벡터 검색
k = 5
distances, indices = index.search(query_np, k)

print("유사한 문장 인덱스:", indices)
print("거리:", distances)

# 원래 문장 매핑
for idx in indices[0]:
    print("유사한 문장:", sentences[idx])

유사한 문장 인덱스: [[ 0  1 10  6  7]]
거리: [[0.9217967 1.392465  1.627958  1.6433356 1.7773175]]
유사한 문장: The weather is lovely today.
유사한 문장: It's so sunny outside!
유사한 문장: She painted a colorful sunset on the wide canvas
유사한 문장: The sun rises early in the quiet mountain village
유사한 문장: He bought fresh bread from the bakery this morning
